In [ ]:
import geopandas as gpd
from shapely.ops import split
import os

# Créer les dossiers de sortie si non existants
os.makedirs("p1", exist_ok=True)
os.makedirs("p2", exist_ok=True)

# Charger la couche de polygones
gdf = gpd.read_file("/content/drive/MyDrive/data_set/kontur/contour.geojson")

# Charger la ligne horizontale de découpe
line = gpd.read_file("/content/line.geojson").geometry.iloc[0]

# Listes pour stocker les morceaux
parts_upper = []
parts_lower = []

for idx, geom in enumerate(gdf.geometry):
    try:
        # Découper le polygone par la ligne
        splitted = split(geom, line)  # retourne une GeometryCollection

        for part in splitted.geoms:
            # On regarde le centroid du morceau pour savoir s'il est au-dessus ou en-dessous de la ligne
            if part.centroid.y >= line.centroid.y:
                parts_upper.append(gpd.GeoDataFrame(gdf.iloc[[idx]], geometry=[part], crs=gdf.crs))
            else:
                parts_lower.append(gpd.GeoDataFrame(gdf.iloc[[idx]], geometry=[part], crs=gdf.crs))

    except Exception as e:
        print(f"Erreur sur polygone {idx}: {e}")
        # Si pas possible de découper, on garde le polygone entier dans le bas
        parts_lower.append(gpd.GeoDataFrame(gdf.iloc[[idx]], geometry=[geom], crs=gdf.crs))

# Fusionner et sauvegarder
gpd.GeoDataFrame(pd.concat(parts_upper, ignore_index=True)).to_file("p1/partie_upper.shp")
gpd.GeoDataFrame(pd.concat(parts_lower, ignore_index=True)).to_file("p2/partie_lower.shp")


/tmp/ipython-input-153699141.py:37: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(pd.concat(parts_upper, ignore_index=True)).to_file("p1/partie_upper.shp")
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'shapegroup_2' to 'shapegro_1'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'shapetype_2' to 'shapetype_'
  ogr_write(
/tmp/ipython-input-153699141.py:38: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(pd.concat(parts_lower, ignore_index=True)).to_file("p2/partie_lower.shp")
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'shapegroup_2' to 'shapegro_1'
  ogr_write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normaliz

In [ ]:
pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 91.8 MB/s eta 0:00:00


In [ ]:
import rasterio
import numpy as np
import glob
import os

# === Dossier racine contenant les sous-dossiers ===
base_dir = "/content/drive/MyDrive/data_set/climate/algeria_tunisia_climate"   # adapte si besoin

# === Dossiers et motifs de fichiers ===
folders = {
    "tmin": {"path": os.path.join(base_dir, "tmin"), "pattern": "Tmin_*_clipped.tif"},
    "tmax": {"path": os.path.join(base_dir, "tmax"), "pattern": "Tmax_*_clipped.tif"},
    "precip": {"path": os.path.join(base_dir, "precipitation"), "pattern": "Precipitation_*_clipped.tif"},
}

# === Dossier de sortie ===
output_dir = os.path.join(base_dir, "output_saisons")
os.makedirs(output_dir, exist_ok=True)

# === Définition des saisons ===
saisons = {
    "s1": [1, 2, 3],     # Jan - Mar
    "s2": [4, 5, 6],     # Apr - Jun
    "s3": [7, 8, 9],     # Jul - Sep
    "s4": [10, 11, 12],  # Oct - Dec
}

# === Étape 1 : calcul des moyennes saisonnières ===
seasonal_files = []

for var, info in folders.items():
    folder = info["path"]
    pattern = info["pattern"]
    print(f"\n--- Traitement de {var.upper()} ---")

    # Recherche des fichiers
    files = sorted(glob.glob(os.path.join(folder, pattern)))
    if len(files) != 12:
        raise ValueError(f"❌ {var}: trouvé {len(files)} fichiers au lieu de 12 dans {folder}")

    arrays = []
    profile = None

    for f in files:
        with rasterio.open(f) as src:
            data = src.read(1).astype(float)
            arrays.append(data)
            if profile is None:
                profile = src.profile

    arrays = np.stack(arrays)  # (12, H, W)

    # Moyennes saisonnières
    for saison, mois in saisons.items():
        indices = [m - 1 for m in mois]
        season_mean = np.nanmean(arrays[indices, :, :], axis=0)

        out_path = os.path.join(output_dir, f"{var}_{saison}.tif")
        profile.update(dtype=rasterio.float32, count=1)

        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(season_mean.astype(rasterio.float32), 1)

        seasonal_files.append(out_path)
        print(f"✅ {out_path} enregistré.")

print("\n✔️ Étape 1 terminée : moyennes saisonnières générées.\n")

# === Étape 2 : création du raster multibande ===
final_order = [
    "tmin_s1", "tmin_s2", "tmin_s3", "tmin_s4",
    "tmax_s1", "tmax_s2", "tmax_s3", "tmax_s4",
    "precip_s1", "precip_s2", "precip_s3", "precip_s4"
]

ordered_files = []
for name in final_order:
    matches = [f for f in seasonal_files if os.path.basename(f).startswith(name)]
    if not matches:
        raise FileNotFoundError(f"Fichier manquant pour {name}")
    ordered_files.append(matches[0])

# Charger le profil du premier raster
with rasterio.open(ordered_files[0]) as src:
    profile = src.profile
profile.update(count=len(ordered_files), dtype=rasterio.float32)

# Fusion des 12 bandes
output_all = os.path.join(output_dir, "saisons_12bandes.tif")
with rasterio.open(output_all, "w", **profile) as dst:
    for i, f in enumerate(ordered_files, start=1):
        with rasterio.open(f) as src:
            dst.write(src.read(1), i)
            print(f"Ajout bande {i}: {os.path.basename(f)}")

print(f"\n🎉 Raster multibande créé avec succès : {output_all}")
print("Bandes = [tmin_s1..4, tmax_s1..4, precip_s1..4]")



--- Traitement de TMIN ---
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmin_s1.tif enregistré.
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmin_s2.tif enregistré.


/tmp/ipython-input-357244031.py:56: RuntimeWarning: Mean of empty slice
  season_mean = np.nanmean(arrays[indices, :, :], axis=0)


✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmin_s3.tif enregistré.
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmin_s4.tif enregistré.

--- Traitement de TMAX ---
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmax_s1.tif enregistré.
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmax_s2.tif enregistré.
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmax_s3.tif enregistré.
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/tmax_s4.tif enregistré.

--- Traitement de PRECIP ---
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/precip_s1.tif enregistré.
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/precip_s2.tif enregistré.
✅ /content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/precip_s3.tif enr

In [ ]:
import rasterio
from rasterio import features
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
from shapely.geometry import shape
from scipy.ndimage import generic_filter

def fill_nan_with_neighbors(arr):
    def filter_func(values):
        center = values[len(values)//2]
        if np.isnan(center):
            neighbors = values[values == values]
            if len(neighbors) > 0:
                return np.mean(neighbors)
            else:
                return 0
        else:
            return center
    return generic_filter(arr, filter_func, size=3, mode='constant', cval=np.nan)

raster_path ="/content/drive/MyDrive/data_set/climate/algeria_tunisia_climate/output_saisons/saisons_12bandes.tif"
shapefile_path = "resultat.shp"

polygons = []

with rasterio.open(raster_path) as src:
    data = src.read().astype(float)  # (12, H, W)
    transform = src.transform
    crs = src.crs

    # Remplir les NaN pour chaque bande
    for b in range(data.shape[0]):
        data[b] = fill_nan_with_neighbors(data[b])

    mask_array = np.ones(data.shape[1:], dtype=bool)

    # Polygonisation de la première bande
    for geom, val in features.shapes(data[0], mask=mask_array, transform=transform):
        poly_attr = {}
        for b in range(data.shape[0]):
            out_image, _ = mask(src, [geom], crop=True, filled=True, indexes=b+1)
            band_data = out_image[0]
            poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
        polygons.append({"geometry": shape(geom), **poly_attr})

# Créer le GeoDataFrame et sauvegarder
gdf = gpd.GeoDataFrame(polygons, crs=crs)
gdf.to_file(shapefile_path)

print(f"Shapefile créé avec {len(gdf)} polygones et {data.shape[0]} bandes comme attributs.")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
/tmp/ipython-input-1842922734.py:44: RuntimeWarning: Mean of empty slice
  poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
/tmp/ipython-input-1842922734.py:44: RuntimeWarning: Mean of empty slice
  poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
/tmp/ipython-input-1842922734.py:44: RuntimeWarning: Mean of empty slice
  poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
/tmp/ipython-input-1842922734.py:44: RuntimeWarning: Mean of empty slice
  poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
/tmp/ipython-input-1842922734.py:44: RuntimeWarning: Mean of empty slice
  poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
/tmp/ipython-input-1842922734.py:44: RuntimeWarning: Mean of empty slice
  poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
/tmp/ipython-input-1842922734.py:44: RuntimeWarning: Mean of empty slice
  poly_attr[f"Band{b+1}"] = float(np.nanmean(band_data))
/tmp/ipython

Shapefile créé avec 15080 polygones et 12 bandes comme attributs.


In [16]:
import geopandas as gpd

# === Charger tes shapefiles ===
# couche à découper
shp = gpd.read_file("/content/drive/MyDrive/data_set/landcover/land_cover_dzd_tun/dzd_tun_landcover.shp")

# couche utilisée pour découper (ligne ou polygone)
clip_layer = gpd.read_file("/content/drive/MyDrive/data_set/clipped_final.geojson")

# === Effectuer le clip ===
# si c’est une ligne, tu peux bufferiser un peu avant (pour avoir une zone)
# clip_layer = clip_layer.buffer(0.0001)

result = gpd.clip(shp, clip_layer)

# === Sauvegarder le résultat ===
result.to_file("/content/drive/MyDrive/data_set/landcover/land_cover_dzd_tun/clipped_landcover.shp")

print("✅ Découpage terminé et shapefile sauvegardé !")


KeyboardInterrupt: 